# Ball Challenge

Predicting the probability of impact given a three-channel timeseries of acceleration data (x-, y-, z-direction)
- Input: Three-channel acceleration data
    - Assumption: Sampled at 100 Hz and recorded for 3 seconds
- Output: Two-dimensional probability map containing the probability of impact for each cell in a grid
    - Assumption: 10x10 grid

In [ ]:
from dataclasses import dataclass

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from elasticai.creator.base_modules.arithmetics.fixed_point_arithmetics import FixedPointArithmetics
from elasticai.creator.base_modules.two_complement_fixed_point_config import FixedPointConfig
from elasticai.creator.nn import Sequential as TranslatableSequential
from elasticai.creator.nn import FPBatchNormedConv1d, FPLinear, FPHardTanh

In [ ]:
@dataclass(frozen=True)
class FixedPointRepresentation:
    total_bits: int
    frac_bits: int

    @property
    def args(self) -> dict[str, int]:
        return dict(total_bits=self.total_bits, frac_bits=self.frac_bits)

    @property
    def arithmetics(self) -> FixedPointArithmetics:
        return FixedPointArithmetics(config=FixedPointConfig(**self.args))

In [ ]:
class ProbabilityMapSoftmax(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
    
    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        *first_dims, height, width = inputs.shape
        vector = inputs.view(*first_dims, height * width)
        probability_vector = F.softmax(vector, dim=len(first_dims))
        probability_map = probability_vector.view(*first_dims, height, width)
        return probability_map

In [ ]:
class BallChallengeModel(torch.nn.Module):
    def __init__(
        self,
        signal_length: int,
        impact_grid_size: tuple[int, int],
        fixed_point_representation: FixedPointRepresentation
    ) -> None:
        super().__init__()
        self.grid_width, self.grid_height = impact_grid_size
        self.fp_repr = fixed_point_representation
        
        self.hardware_model = TranslatableSequential(
            FPBatchNormedConv1d(
                in_channels=3,
                out_channels=32,
                signal_length=signal_length,
                kernel_size=8,
                **self.fp_repr.args
            ),
            FPHardTanh(**self.fp_repr.args),
            FPBatchNormedConv1d(
                in_channels=32,
                out_channels=4,
                signal_length=signal_length - 9,
                kernel_size=4,
                **self.fp_repr.args
            ),
            FPHardTanh(**self.fp_repr.args),
            FPLinear(
                in_features=(signal_length - 14) * 4,
                out_features=self.grid_width * self.grid_height,
                bias=True,
                **self.fp_repr.args
            ),
        )
        self.softmax = ProbabilityMapSoftmax()
    
    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        quantized_inputs = self.fp_repr.arithmetics.quantize(inputs)
        predictions = self.hardware_model(quantized_inputs)
        
        *first_dims, _ = predictions.shape
        prediction_map = predictions.view(*first_dims, self.grid_height, self.grid_width)

        return self.softmax(prediction_map)        

In [ ]:
fixed_point_repr = FixedPointRepresentation(total_bits=16, frac_bits=8)

sampling_freq = 100
recording_time = 3

model = BallChallengeModel(
    signal_length=sampling_freq * recording_time,
    impact_grid_size=(10, 10),
    fixed_point_representation=fixed_point_repr,
)

random_inputs = torch.randn(64, 3, sampling_freq * recording_time)

model(random_inputs)